## Dependency Parsing as a Preprocessing Step for Logical Reasoning

In [5]:
# Installs HBOX for Jupyer Notebooks
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter nbextension enable --py widgetsnbextension
# drive_path = ''

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/FinalProject/'
!pip install transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import gc
import re
import json
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
#from transformers import RobertaTokenizer
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
#from transformers import RobertaConfig, RobertaForSequenceClassification, AdamW
from transformers import BertConfig, BertForSequenceClassification, AdamW

# Enable CUDA Blocking Debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
CUDA_LAUNCH_BLOCKING="1"

# Print GPU Information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Wed Dec 15 22:54:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Load Data

In [7]:
def load_data(path, test):
    df = pd.DataFrame({'prompt': [], 'label': []})
    data = json.load(open(path))
    for i in range(len(data['context'])):
        # Add BERT tokens to prompt
        prompt = data['context'][str(i)] + '[SEP]' + data['question'][str(i)] + '[SEP]'# + data['dep_context'][str(i)] + ' </s> '
        for j in range(4):
            # Add BERT tokens to answer
            answer = data['answers'][str(i)][j]# + ' <d> ' + data['dep_answers'][str(i)][j] + ' <s>'
            
            # Attach 0, 1 label as array
            label = [1] if not test and j == data['label'][str(i)] else [0]
            
            # Append question, answer pair to dataframe
            df = df.append({'prompt': prompt + answer, 'label': [label]}, ignore_index=True)            
    return df

In [8]:
train_data = load_data(drive_path + 'reclor_data_with_dependencies/train.json', False)
val_data = load_data(drive_path + 'reclor_data_with_dependencies/val.json', False)
test_data = load_data(drive_path + 'reclor_data_with_dependencies/test.json', True)

In [9]:
# Describe the token lengths of training data
train_data['prompt'].apply(lambda x: len(re.findall(r'\w+', x))).describe()

count    18552.000000
mean       103.285630
std         23.085243
min         36.000000
25%         88.000000
50%        102.000000
75%        116.000000
max        230.000000
Name: prompt, dtype: float64

#### Initialize tokenizer

In [10]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

#### Tokenize the data

In [11]:
# Creates a dataloader (which includes an attention mask)
def preprocess(in_, tokenizer, max_len, batch_size, data_class='train'):
    encoded_input = tokenizer(in_['prompt'].values.tolist(), padding=True, max_length=max_len, truncation=True, return_tensors="pt")
    
    if data_class != 'test':
        labels = torch.tensor(in_['label'].values.tolist())
    dataset_tensor = TensorDataset(encoded_input['input_ids'], encoded_input['attention_mask'], labels)
    sampler = SequentialSampler(dataset_tensor)
    #sampler = RandomSampler(dataset_tensor) if data_class == "train" else SequentialSampler(dataset_tensor)
    dataloader = DataLoader(dataset_tensor, sampler=sampler, batch_size=batch_size)
    return dataloader

In [12]:
max_len = 512 # should be 1024
batch_size = 4

train_dataloader = preprocess(train_data, tokenizer, max_len, batch_size)
val_dataloader = preprocess(val_data, tokenizer, max_len, batch_size, data_class="val")
# test_dataloader = preprocess(test_data, tokenizer, max_len, batch_size, data_class="test")

In [13]:
# fi = iter(val_dataloader)
# for i in range(100):
#   l = next(fi)
#   #print(l)
#   #print(l.numpy().tolist()[0])
#   k = tokenizer.decode(l.numpy().tolist()[0])
#   print(k)
#   j = tokenizer.decode(l.numpy().tolist()[1])
#   print(j)
#   print(l)
# print(next(iter(val_dataloader)))
# print(train_data)

# fi = iter(val_dataloader)
# for i in range(10):
#   l = next(fi)
#   print(l)
#   print(tokenizer.decode(l[0]))
#   print(tokenizer.decode(l[1]))


for step, batch in enumerate(val_dataloader):
  labels = batch[0]
  print(tokenizer.decode(labels.numpy().tolist()[0]))
  print(tokenizer.decode(labels.numpy().tolist()[1]))

[CLS] in a business whose owners and employees all belong to one family, the employees can be paid exceptionally low wages. hence, general operating expenses are much lower than they would be for other business ventures, making profits higher. so a family business is a family's surest road to financial prosperity. [SEP] the reasoning in the argument is flawed because the argument [SEP] ignores the fact that in a family business, paying family members low wages may itself reduce the family's prosperity [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

#### Train and Evaluate RoBERTa

In [14]:
def ClearTorch():
    torch.no_grad()
    torch.cuda.empty_cache()
    gc.collect()
    
def Eval(model, dataloader):
    ClearTorch()
    model.eval()
    predictions, true_labels = [], []
    for step, batch in enumerate(tqdm(dataloader)):
        # Call model on batch
        input_ids, attention_mask, labels = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        # Convert output logit to predictions using softmax
        #print(labels)

        predictions.append(torch.nn.functional.softmax(outputs.logits).argmax(0)[1].cpu().numpy().tolist())
        true_labels.append(labels.argmax(0).cpu().numpy().tolist()[0][0])
      
        ClearTorch()

    return float(sum([predictions[i] == true_labels[i] for i in range(len(predictions)) ])) / float(len(predictions))

def Train(model, train_data, lr, n_epoch):
    ClearTorch()
    optimizer = AdamW(model.parameters(), lr=lr)

    for epoch in range(n_epoch):
        print(f"Epoch {epoch}")
        model.train()
        nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0

        for step, batch in enumerate(tqdm(train_data)):
            # RoBERTa fine-tuning
            input_ids, attention_mask, labels = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            outputs.loss.backward()
            optimizer.step()
            
            ClearTorch()
            
            tr_loss += float(outputs.loss)
            nb_tr_steps += 1
            
        print(f"Train loss on epoch {epoch}: {tr_loss / nb_tr_steps}\n")

In [15]:
ClearTorch()

config = BertConfig.from_pretrained('bert-base-uncased')
# config.max_position_embeddings = max_len
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    model.to(device)
    model = torch.nn.DataParallel(model)
else:
    model.cuda()
    
learning_rate = 2e-4
num_epoch = 1

Train(model, train_dataloader, learning_rate, num_epoch)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 0


100%|██████████| 4638/4638 [30:20<00:00,  2.55it/s]

Train loss on epoch 0: 0.5686955137363844



In [16]:
val_dataloader = preprocess(val_data, tokenizer, max_len, batch_size)#, data_class="val")
print(f"Accuracy: {Eval(model, val_dataloader)}")

100%|██████████| 500/500 [01:47<00:00,  4.64it/s]

Accuracy: 0.234
